In [1]:
import os
import sys
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [2]:
sys.path.append("../src/")

In [3]:
from utilities import *

In [4]:
pd.set_option("display.max_columns", None)

In [5]:
def read_parquet(identifier):
    filepath = os.path.join(dir_dict["clean"], f"{identifier}.parquet")
    return pd.read_parquet(filepath)

In [6]:
def merge_datasets(identifier):
    events = read_parquet(f"{identifier}_events")
    fights = read_parquet(f"{identifier}_fights")
    fighters = read_parquet("fighters")
    
    df = events.merge(fights, on="event_id", how="right").merge(fighters, on="fighter_id", how="left")
    df = df.drop(["event_id"], axis=1)

    general_cols = [col for col in df.columns \
                    if not (col.startswith("fight_fighter_") or col.startswith("fighter_") or col == "fight_id")]

    df = df.merge(df.drop(general_cols, axis=1), on="fight_id", suffixes=("_A", "_B"))

    df = df.loc[df["fighter_id_A"] != df["fighter_id_B"]].reset_index(drop=True)

    filepath = os.path.join(dir_dict["clean"], f"{identifier}_merged.parquet")
    df.to_parquet(filepath)
    
    return df

In [7]:
def merge_completed_upcoming():
    completed = merge_datasets("completed")
    upcoming = merge_datasets("upcoming")
    
    return completed, upcoming

In [8]:
completed, upcoming = merge_completed_upcoming()

In [9]:
completed

,event_name,event_date,event_city,event_state,event_country,fight_id,fight_bout,fight_method,fight_round,fight_time_format,fight_referee,fight_details,fight_fighter_overall_kd_A,fight_fighter_round1_kd_A,fight_fighter_round2_kd_A,fight_fighter_overall_sig_str_%_A,fight_fighter_round1_sig_str_%_A,fight_fighter_round2_sig_str_%_A,fight_fighter_overall_td_%_A,fight_fighter_round1_td_%_A,fight_fighter_round2_td_%_A,fight_fighter_overall_sub_att_A,fight_fighter_round1_sub_att_A,fight_fighter_round2_sub_att_A,fight_fighter_overall_rev_A,fight_fighter_round1_rev_A,fight_fighter_round2_rev_A,fight_fighter_overall_sig_str_%_ss_A,fight_fighter_round1_sig_str_%_ss_A,fight_fighter_round2_sig_str_%_ss_A,fight_fighter_round3_kd_A,fight_fighter_round3_sig_str_%_A,fight_fighter_round3_td_%_A,fight_fighter_round3_sub_att_A,fight_fighter_round3_rev_A,fight_fighter_round3_sig_str_%_ss_A,fight_fighter_round4_kd_A,fight_fighter_round4_sig_str_%_A,fight_fighter_round4_td_%_A,fight_fighter_round4_sub_att_A,fight_fighter_round4_rev_A,fight_fighter_round4_sig_str_%_ss_A,fight_fighter_round5_kd_A,fight_fighter_round5_sig_str_%_A,fight_fighter_round5_td_%_A,fight_fighter_round5_sub_att_A,fight_fighter_round5_rev_A,fight_fighter_round5_sig_str_%_ss_A,fight_weight_class,fighter_id_A,fight_fighter_won_A,fight_fighter_overall_sig_str_landed_A,fight_fighter_overall_sig_str_total_A,fight_fighter_round1_sig_str_landed_A,fight_fighter_round1_sig_str_total_A,fight_fighter_round2_sig_str_landed_A,fight_fighter_round2_sig_str_total_A,fight_fighter_overall_total_str_landed_A,fight_fighter_overall_total_str_total_A,fight_fighter_round1_total_str_landed_A,fight_fighter_round1_total_str_total_A,fight_fighter_round2_total_str_landed_A,fight_fighter_round2_total_str_total_A,fight_fighter_overall_td_landed_A,fight_fighter_overall_td_total_A,fight_fighter_round1_td_landed_A,fight_fighter_round1_td_total_A,fight_fighter_round2_td_landed_A,fight_fighter_round2_td_total_A,fight_fighter_overall_sig_str_ss_landed_A,fight_fighter_overall_sig_str_ss_total_A,fight_fighter_round1_sig_str_ss_landed_A,fight_fighter_round1_sig_str_ss_total_A,fight_fighter_round2_sig_str_ss_landed_A,fight_fighter_round2_sig_str_ss_total_A,fight_fighter_overall_head_ss_landed_A,fight_fighter_overall_head_ss_total_A,fight_fighter_round1_head_ss_landed_A,fight_fighter_round1_head_ss_total_A,fight_fighter_round2_head_ss_landed_A,fight_fighter_round2_head_ss_total_A,fight_fighter_overall_body_ss_landed_A,fight_fighter_overall_body_ss_total_A,fight_fighter_round1_body_ss_landed_A,fight_fighter_round1_body_ss_total_A,fight_fighter_round2_body_ss_landed_A,fight_fighter_round2_body_ss_total_A,fight_fighter_overall_leg_ss_landed_A,fight_fighter_overall_leg_ss_total_A,fight_fighter_round1_leg_ss_landed_A,fight_fighter_round1_leg_ss_total_A,fight_fighter_round2_leg_ss_landed_A,fight_fighter_round2_leg_ss_total_A,fight_fighter_overall_distance_ss_landed_A,fight_fighter_overall_distance_ss_total_A,fight_fighter_round1_distance_ss_landed_A,fight_fighter_round1_distance_ss_total_A,fight_fighter_round2_distance_ss_landed_A,fight_fighter_round2_distance_ss_total_A,fight_fighter_overall_clinch_ss_landed_A,fight_fighter_overall_clinch_ss_total_A,fight_fighter_round1_clinch_ss_landed_A,fight_fighter_round1_clinch_ss_total_A,fight_fighter_round2_clinch_ss_landed_A,fight_fighter_round2_clinch_ss_total_A,fight_fighter_overall_ground_ss_landed_A,fight_fighter_overall_ground_ss_total_A,fight_fighter_round1_ground_ss_landed_A,fight_fighter_round1_ground_ss_total_A,fight_fighter_round2_ground_ss_landed_A,fight_fighter_round2_ground_ss_total_A,fight_fighter_round3_sig_str_landed_A,fight_fighter_round3_sig_str_total_A,fight_fighter_round3_total_str_landed_A,fight_fighter_round3_total_str_total_A,fight_fighter_round3_td_landed_A,fight_fighter_round3_td_total_A,fight_fighter_round3_sig_str_ss_landed_A,fight_fighter_round3_sig_str_ss_total_A,fight_fighter_round3_head_ss_landed_A,fight_fighter_round3_head_ss_total_A,fight_fighter_roun

In [10]:
upcoming

,event_name,event_date,event_city,event_state,event_country,fight_id,bout,weight_class,fighter_id_A,fighter_name_A,fighter_nickname_A,fighter_stance_A,fighter_dob_A,fighter_slpm_A,fighter_str_acc_A,fighter_sapm_A,fighter_str_def_A,fighter_td_avg_A,fighter_td_acc_A,fighter_td_def_A,fighter_sub_avg_A,fighter_wins_A,fighter_losses_A,fighter_ties_A,fighter_height_inches_A,fighter_reach_inches_A,fighter_weight_lbs_A,fighter_id_B,fighter_name_B,fighter_nickname_B,fighter_stance_B,fighter_dob_B,fighter_slpm_B,fighter_str_acc_B,fighter_sapm_B,fighter_str_def_B,fighter_td_avg_B,fighter_td_acc_B,fighter_td_def_B,fighter_sub_avg_B,fighter_wins_B,fighter_losses_B,fighter_ties_B,fighter_height_inches_B,fighter_reach_inches_B,fighter_weight_lbs_B
0,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,2040680412342412658,Welterweight Bout,Welterweight,2522666563856122122,Gilbert Burns,Durinho,Orthodox,1986-07-20,3.46,0.48,3.21,0.54,2.00,0.34,0.47,0.6,20,5,0,70.0,71.0,170.0,9562241329663913669,Neil Magny,The Haitian Sensation,Orthodox,1987-08-03,3.62,0.46,2.27,0.54,2.43,0.42,0.57,0.3,27,10,0,75.0,80.0,170.0
1,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,2040680412342412658,Welterweight Bout,Welterweight,2522666563856122122,Gilbert Burns,Durinho,Orthodox,1986-07-20,3.46,0.48,3.21,0.54,2.00,0.34,0.47,0.6,20,5,0,70.0,71.0,170.0,9562241329663913669,Neil Magny,The Haitian Sensation,Orthodox,1987-08-03,3.62,0.46,2.27,0.54,2.43,0.42,0.57,0.3,27,10,0,75.0,80.0,170.0
2,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,2040680412342412658,Welterweight Bout,Welterweight,2522666563856122122,Gilbert Burns,Durinho,Orthodox,1986-07-20,3.46,0.48,3.21,0.54,2.00,0.34,0.47,0.6,20,5,0,70.0,71.0,170.0,9562241329663913669,Neil Magny,The Haitian Sensation,Orthodox,1987-08-03,3.62,0.46,2.27,0.54,2.43,0.42,0.57,0.3,27,10,0,75.0,80.0,170.0
3,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,2040680412342412658,Welterweight Bout,Welterweight,2522666563856122122,Gilbert Burns,Durinho,Orthodox,1986-07-20,3.46,0.48,3.21,0.54,2.00,0.34,0.47,0.6,20,5,0,70.0,71.0,170.0,9562241329663913669,Neil Magny,The Haitian Sensation,Orthodox,1987-08-03,3.62,0.46,2.27,0.54,2.43,0.42,0.57,0.3,27,10,0,75.0,80.0,170.0
4,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,2040680412342412658,Welterweight Bout,Welterweight,9562241329663913669,Neil Magny,The Haitian Sensation,Orthodox,1987-08-03,3.62,0.46,2.27,0.54,2.43,0.42,0.57,0.3,27,10,0,75.0,80.0,170.0,2522666563856122122,Gilbert Burns,Durinho,Orthodox,1986-07-20,3.46,0.48,3.21,0.54,2.00,0.34,0.47,0.6,20,5,0,70.0,71.0,170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,153571164976329024,Women's Flyweight Bout,Women's Flyweight,8258840672665930391,Lauren Murphy,Lucky,Orthodox,1983-07-27,3.84,0.40,4.45,0.58,1.11,0.34,0.71,0.1,16,5,0,65.0,67.0,125.0,7645143793069688591,Jessica Andrade,Bate Estaca,Orthodox,1991-09-25,6.16,0.50,5.16,0.53,2.85,0.55,0.66,0.5,23,9,0,61.0,62.0,125.0
228,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,153571164976329024,Women's Flyweight Bout,Women's Flyweight,7645143793069688591,Jessica Andrade,Bate Estaca,Orthodox,1991-09-25,6.16,0.50,5.16,0.53,2.85,0.55,0.66,0.5,23,9,0,61.0,62.0,125.0,8258840672665930391,Lauren Murphy,Lucky,Orthodox,1983-07-27,3.84,0.40,4.45,0.58,1.11,0.34,0.71,0.1,16,5,0,65.0,67.0,125.0
229,UFC 283: Teixeira vs. Hill,2023-01-21,Rio de Janeiro,Rio de Janeiro,Brazil,153571164976329024,Women's Flyweight Bout,Women's Flyweight,7645143793069688591,Jessica Andrade,Bate Estaca,Orthodox,1991-09-25,6.16,0.50,5.16,0.53,2.85,0.55,0.66,0.5,23,9,0,61.0,62.0,125.0,8258840672665930391,Lauren Murphy,Lucky,Orthodox,1983-07-27,3.84,0.40,4.45,0.5